In [1]:
#|hide
from fastrl.test_utils import initialize_notebook
initialize_notebook()

In [2]:
#|default_exp agents.dqn.basic

In [3]:
#|export
# Python native modules
import os
from collections import deque
from typing import Callable,Optional,List
# Third party libs
from fastcore.all import ifnone
import torchdata.datapipes as dp
from torchdata.dataloader2 import DataLoader2
from torchdata.dataloader2.graph import traverse_dps,DataPipe
import torch
import torch.nn.functional as F
from torch import optim
from torch import nn
import numpy as np
# Local modules
# from fastrl.core import *
from fastrl.agents.core import AgentHead,AgentBase
from fastrl.pipes.core import find_dp
# from fastrl.data.block import *
# from fastrl.dataloader2_ext import *
from fastrl.memory.experience_replay import ExperienceReplay
from fastrl.agents.core import StepFieldSelector,SimpleModelRunner,NumpyConverter
from fastrl.agents.discrete import EpsilonCollector,PyPrimativeConverter,ArgMaxer,EpsilonSelector
from fastrl.loggers.core import (
    CacheLoggerBase,LogCollector,Record,LoggerBasePassThrough,BatchCollector,EpocherCollector,RollingTerminatedRewardCollector,EpisodeCollector
)
# from fastrl.loggers.vscode_visualizers import *
from fastrl.learner.core import LearnerBase,LearnerHead,StepBatcher
from fastrl.torch_core import Module
# from fastrl.data.dataloader2 import *

# DQN Basic
> Core DQN modules, pipes, and tooling

## Model

In [4]:
#|export
class DQN(Module):
    def __init__(self,
                 state_sz:int,  # The input dim of the state
                 action_sz:int, # The output dim of the actions
                 hidden=512,    # Number of neurons connected between the 2 input/output layers
                 head_layer:Module=nn.Linear, # DQN extensions such as Dueling DQNs have custom heads
                 activition_fn:Module=nn.ReLU # The activiation fn used by `DQN`
                ):
        self.layers=nn.Sequential(
            nn.Linear(state_sz,hidden),
            activition_fn(),
            head_layer(hidden,action_sz),
        )
    def forward(self,x): return self.layers(x)


## Agent

In [5]:
#|export
DataPipeAugmentationFn = Callable[[DataPipe],Optional[DataPipe]]

def DQNAgent(
    model,
    logger_bases=None,
    min_epsilon=0.02,
    max_epsilon=1,
    max_steps=1000,
    device='cpu',
    dp_augmentation_fns:Optional[List[DataPipeAugmentationFn]]=None
)->AgentHead:
    agent_base = AgentBase(model,logger_bases=ifnone(logger_bases,[CacheLoggerBase()]))
    agent = StepFieldSelector(agent_base,field='state')
    # agent = InputInjester(agent)
    agent = SimpleModelRunner(agent).to(device=device)
    agent = ArgMaxer(agent)
    agent = EpsilonSelector(agent,min_epsilon=min_epsilon,max_epsilon=max_epsilon,max_steps=max_steps,device=device)
    if agent_base.logger_bases is not None: 
        agent = EpsilonCollector(agent)
    agent = ArgMaxer(agent,only_idx=True)
    agent = NumpyConverter(agent)
    agent = PyPrimativeConverter(agent)
    agent = AgentHead(agent)
    
    for fn in ifnone(dp_augmentation_fns,[]):
        result = fn(agent)
        if result is not None: agent = result
    return agent

In [6]:
torch.manual_seed(0)
model = DQN(4,2)

agent = DQNAgent(model)

In [7]:
from fastcore.all import test_eq
from fastrl.core import SimpleStep

In [8]:
input_tensor = torch.tensor([1,2,3,4]).float()
step = SimpleStep(state=input_tensor)

for action in agent([step]):
    print(action)
    
test_eq(input_tensor,torch.tensor([1., 2., 3., 4.]))

1


In [9]:
from fastrl.envs.gym import GymDataPipe
from fastrl.loggers.core import ProgressBarLogger

In [10]:
AgentHead.debug=True

In [11]:
# Setup Logger
logger_base = ProgressBarLogger()

# Setup up the core NN
torch.manual_seed(0)
model = DQN(4,2)

agent = DQNAgent(model,[logger_base])

pipes = GymDataPipe(['CartPole-v1']*1,agent=agent,n=10)
# dls = L(block.dataloaders(['CartPole-v1']*1,n=10,bs=1))
# pipes = list(block(['CartPole-v1']*1))

# list(dls[0])
list(pipes)
traverse_dps(agent)

/usr/local/lib/python3.8/dist-packages/torchdata/datapipes/iter/util/header.py:57: UserWarning: The length of this HeaderIterDataPipe is inferred to be equal to its limit.The actual value may be smaller if the actual length of source_datapipe is smaller than the limit.
  warn(
/home/fastrl_user/fastrl/fastrl/envs/gym.py:121: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),


{140251137503392: (AgentHead,
  {140251137503344: (PyPrimativeConverter,
    {140251137970080: (NumpyConverter,
      {140251137968016: (ArgMaxer,
        {140251137969840: (EpsilonCollector,
          {140251137969888: (EpsilonSelector,
            {140251137969456: (ArgMaxer,
              {140251137969360: (SimpleModelRunner,
                {140251340554784: (StepFieldSelector,
                  {140253584956192: (AgentBase,
                    {140251137969264: (ProgressBarLogger, {})})}),
                 140253584956192: (AgentBase,
                  {140251137969264: (ProgressBarLogger, {})})})}),
             140253584956192: (AgentBase,
              {140251137969264: (ProgressBarLogger, {})})})})})})}),
   140253584956192: (AgentBase, {140251137969264: (ProgressBarLogger, {})})})}

## Training DataPipes

In [12]:
#|export
class QCalc(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe):
        self.source_datapipe = source_datapipe
        
    def __iter__(self):
        self.learner = find_dp(traverse_dps(self),LearnerBase)
        for batch in self.source_datapipe:
            self.learner.done_mask = batch.terminated.reshape(-1,)
            self.learner.next_q = self.learner.model(batch.next_state)
            self.learner.next_q = self.learner.next_q.max(dim=1).values.reshape(-1,1)
            self.learner.next_q[self.learner.done_mask] = 0 
            yield batch

In [13]:
#|export
class TargetCalc(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe,discount=0.99,nsteps=1):
        self.source_datapipe = source_datapipe
        self.discount = discount
        self.nsteps = nsteps
        self.learner = None
        
    def __iter__(self):
        self.learner = find_dp(traverse_dps(self),LearnerBase)
        for batch in self.source_datapipe:
            self.learner.targets = batch.reward+self.learner.next_q*(self.discount**self.nsteps)
            self.learner.pred = self.learner.model(batch.state)
            self.learner.target_qs = self.learner.pred.clone().float()
            self.learner.target_qs.scatter_(1,batch.action.long(),self.learner.targets.float())
            yield batch

In [14]:
#|export
class LossCalc(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe,discount=0.99,nsteps=1):
        self.source_datapipe = source_datapipe
        self.discount = discount
        self.nsteps = nsteps
        self.learner = find_dp(traverse_dps(self),LearnerBase)
        
    def __iter__(self):
        for batch in self.source_datapipe:
            self.learner.loss_grad = self.learner.loss_func(self.learner.pred, self.learner.target_qs)
            yield batch

In [15]:
#|export
class ModelLearnCalc(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe):
        self.source_datapipe = source_datapipe
        
    def __iter__(self):
        self.learner = find_dp(traverse_dps(self),LearnerBase)
        for batch in self.source_datapipe:
            self.learner.loss_grad.backward()
            self.learner.opt.step()
            self.learner.opt.zero_grad()
            self.learner.loss = self.learner.loss_grad.clone()
            yield self.learner.loss

In [16]:
#|export
class LossCollector(LogCollector):
    title:str='loss'

    def __init__(self,
            source_datapipe, # The parent datapipe, likely the one to collect metrics from
        ):
        self.source_datapipe = source_datapipe
        self.main_buffers = None
        
        
    def __iter__(self):
        self.learner = find_dp(traverse_dps(self),LearnerBase)
        for i,steps in enumerate(self.source_datapipe):
            # if i==0: self.push_header('loss')
            for q in self.main_buffers: q.append(Record('loss',self.learner.loss.cpu().detach().numpy()))
            yield steps

In [17]:
#|export
def DQNLearner(
    model,
    dls,
    logger_bases=(),
    loss_func=nn.MSELoss(),
    opt=optim.AdamW,
    lr=0.005,
    bs=128,
    max_sz=10000,
    nsteps=1,
    device=None,
    batches=None,
    dp_augmentation_fns:Optional[List[DataPipeAugmentationFn]]=None
) -> LearnerHead:
    learner = LearnerBase(model,dls,batches=batches,loss_func=loss_func,opt=opt(model.parameters(),lr=lr))
    learner = LoggerBasePassThrough(learner,logger_bases)
    learner = BatchCollector(learner,batch_on_pipe=LearnerBase)
    learner = EpocherCollector(learner)
    for logger_base in logger_bases: learner = logger_base.connect_source_datapipe(learner)
    if logger_bases: 
        learner = RollingTerminatedRewardCollector(learner)
        learner = EpisodeCollector(learner)
    learner = ExperienceReplay(learner,bs=bs,max_sz=max_sz)
    learner = StepBatcher(learner,device=device)
    learner = QCalc(learner)
    learner = TargetCalc(learner,nsteps=nsteps)
    learner = LossCalc(learner)
    learner = ModelLearnCalc(learner)
    if logger_bases: 
        learner = LossCollector(learner)
    learner = LearnerHead(learner)
    
    for fn in ifnone(dp_augmentation_fns,[]):
        result = fn(learner)
        if result is not None: learner = result
    
    return learner

Try training with basic defaults...

In [18]:
#|eval:false
# Setup Loggers
logger_base = ProgressBarLogger(epoch_on_pipe=EpocherCollector,
                 batch_on_pipe=BatchCollector)

# Setup up the core NN
torch.manual_seed(0)
model = DQN(4,2).cuda()
# Setup the Agent
agent = DQNAgent(model,[logger_base],max_steps=4000,device='cuda')
# Setup the DataBlock
# block = DataBlock(
#     GymTransformBlock(agent=agent,nsteps=1,nskips=1,firstlast=False,bs=1)
# )
dls = GymDataPipe(['CartPole-v1']*1,agent=agent,nsteps=1,nskips=1,firstlast=False,bs=1,as_dataloader=True,num_workers=0)

# Setup the Learner
learner = DQNLearner(model,(dls,),batches=4,#1000,
                     logger_bases=[logger_base],
                      bs=2,#128,
                      max_sz=100_000,device='cuda')
# learner.fit(3)
learner.fit(2)



loss,episode,rolling_reward,epoch,batch
0.49127352,1,None,1,4
0.541654,1,None,1,4


If we try a regular DQN with nsteps/nskips it doesnt really converge after 130. We cant expect stability at all, and im pretty sure that nsteps (correctly) tries to reduce to number of duplicated states so that the agent can sample more unique state transitions. The problem with this is that the base dqn is not stable, so giving it lots of "new" stuff, im not sure helps. In otherwords, its going to forget the old stuff very quickly, and having duplicate states helps "remind it"

In [21]:
logger_base = ProgressBarLogger(epoch_on_pipe=EpocherCollector,
                 batch_on_pipe=BatchCollector)

# Setup up the core NN
torch.manual_seed(0)
model = DQN(4,2)
# Setup the Agent
agent = DQNAgent(model,[logger_base],max_steps=10000)
# Setup the DataBlock
dls = GymDataPipe(['CartPole-v1']*1,agent=agent,nsteps=1,nskips=1,firstlast=False,bs=1,as_dataloader=True,num_workers=0)

# Setup the Learner
learner = DQNLearner(model,(dls,),batches=1000,logger_bases=[logger_base],bs=128,max_sz=20_000,lr=0.001)

# del agent
# learner.fit(3)
learner.fit(20)

KeyboardInterrupt: 

In [ ]:
from fastrl.loggers.vscode_visualizers import VSCodePipeline

In [ ]:
#|hide
#|eval: false
model.eval()

pipe = GymDataPipe(['CartPole-v1']*1,agent=agent,n=100,seed=0,include_images=True)
pipe = VSCodePipeline(pipe)

list(pipe);
pipe.show()

In [ ]:
# Setup Loggers
logger_base = ProgressBarLogger(
    epoch_on_pipe=EpocherCollector,
    batch_on_pipe=BatchCollector
)

# Setup up the core NN
torch.manual_seed(0)
model = DQN(8,4)
# Setup the Agent
agent = DQNAgent(model,[logger_base])
# Setup the DataBlock
dls = GymDataPipe(['LunarLander-v2']*1,agent=agent,n=1000,bs=1,as_dataloader=True,num_workers=0)

# Setup the Learner
learner = DQNLearner(model,(dls,),logger_bases=[logger_base])
learner.fit(3)
# learner.fit(30)

In [ ]:
#|hide
#|eval: false
dls = GymDataPipe(['LunarLander-v2']*1,agent=agent,n=1000,bs=1,as_dataloader=True,include_images=True,num_workers=0)
pipe = VSCodePipeline(pipe)

list(pipe);
pipe.show(step=2)

In [ ]:
#|hide
#|eval: false
!nbdev_export